<a href="https://colab.research.google.com/github/purplepatch-everyday/RecSys/blob/main/Item_Based_Collaborative_Filtering_RecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Building Item Based Collaborative Filtering Recommendation System 
Used
*   Movielens data
*   Cosine similarity



In [2]:
!pip install wget
import wget
import zipfile
import os
import pandas as pd

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=f1e6a596c7868a0b9630b6fefdc4bb2c8240edf928fdb6cb817b8fc20e1e6418
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [1]:
# Google colab mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


If you havn't downloaded movielens data yet, uncomment the code below to download and extract files

In [ ]:
# Download data
# url = "https://files.grouplens.org/datasets/movielens/ml-latest-small.zip"
# wget.download(url,'/content/drive/MyDrive/data')

In [ ]:
# Extract zipfile
# you can change to your own path 

# with zipfile.ZipFile("/content/drive/MyDrive/data/ml-latest-small.zip","r") as zip_ref:
#   zip_ref.extractall("/content/drive/MyDrive/data")

In [3]:
path = "/content/drive/MyDrive/data/ml-latest-small/"
os.listdir(path)

['movies.csv', 'README.txt', 'links.csv', 'ratings.csv', 'tags.csv']

In [19]:
# Read movies,ratings files

ratings_df = pd.read_csv(os.path.join(path, 'ratings.csv'), encoding='utf-8')
movies_df = pd.read_csv(os.path.join(path, 'movies.csv'), index_col='movieId', encoding='utf-8')

In [20]:
print(ratings_df.head())
print(movies_df.head())

   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931
                                      title                                       genres
movieId                                                                                 
1                          Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
2                            Jumanji (1995)                   Adventure|Children|Fantasy
3                   Grumpier Old Men (1995)                               Comedy|Romance
4                  Waiting to Exhale (1995)                         Comedy|Drama|Romance
5        Father of the Bride Part II (1995)                                       Comedy


In [27]:
# Create new df which contains user,movie,ratings data 
print(ratings_df.columns)

# Remove timestamp 
user_movie_df = ratings_df.drop('timestamp',axis=1)
print(user_movie_df.head())

# Merge user_movie_df and movis_df
user_movie_df = pd.merge(left=user_movie_df,right=movies_df,how="left",left_on="movieId",right_on="movieId")
user_movie_df = user_movie_df.drop("genres",axis=1)
print(user_movie_df.head())
print(user_movie_df.columns)



Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')
   userId  movieId  rating
0       1        1     4.0
1       1        3     4.0
2       1        6     4.0
3       1       47     5.0
4       1       50     5.0
   userId  movieId  rating                        title
0       1        1     4.0             Toy Story (1995)
1       1        3     4.0      Grumpier Old Men (1995)
2       1        6     4.0                  Heat (1995)
3       1       47     5.0  Seven (a.k.a. Se7en) (1995)
4       1       50     5.0   Usual Suspects, The (1995)
Index(['userId', 'movieId', 'rating', 'title'], dtype='object')


In [52]:
# Create Pivot table (item based = movie based)

movie_pivot = pd.pivot_table(user_movie_df, index = "title", columns = "userId",values="rating")
movie_pivot.fillna(0,inplace=True)

print(movie_pivot)

userId                                     1    2    3    ...  608  609  610
title                                                     ...               
'71 (2014)                                 0.0  0.0  0.0  ...  0.0  0.0  4.0
'Hellboy': The Seeds of Creation (2004)    0.0  0.0  0.0  ...  0.0  0.0  0.0
'Round Midnight (1986)                     0.0  0.0  0.0  ...  0.0  0.0  0.0
'Salem's Lot (2004)                        0.0  0.0  0.0  ...  0.0  0.0  0.0
'Til There Was You (1997)                  0.0  0.0  0.0  ...  0.0  0.0  0.0
...                                        ...  ...  ...  ...  ...  ...  ...
eXistenZ (1999)                            0.0  0.0  0.0  ...  4.5  0.0  0.0
xXx (2002)                                 0.0  0.0  0.0  ...  3.5  0.0  2.0
xXx: State of the Union (2005)             0.0  0.0  0.0  ...  0.0  0.0  1.5
¡Three Amigos! (1986)                      4.0  0.0  0.0  ...  0.0  0.0  0.0
À nous la liberté (Freedom for Us) (1931)  0.0  0.0  0.0  ...  0.0  0.0  0.0

In [53]:
# Cosine Similarity 

from sklearn.metrics.pairwise import cosine_similarity
movie_cos_sim = cosine_similarity(movie_pivot)
print(movie_cos_sim)


[[1.         0.         0.         ... 0.32732684 0.         0.        ]
 [0.         1.         0.70710678 ... 0.         0.         0.        ]
 [0.         0.70710678 1.         ... 0.         0.         0.        ]
 ...
 [0.32732684 0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [54]:
movie_cos_sim_df = pd.DataFrame(data = movie_cos_sim, index = movie_pivot.index,columns=movie_pivot.index)
print(movie_cos_sim_df.head(2))

title                                    '71 (2014)  ...  À nous la liberté (Freedom for Us) (1931)
title                                                ...                                           
'71 (2014)                                      1.0  ...                                        0.0
'Hellboy': The Seeds of Creation (2004)         0.0  ...                                        0.0

[2 rows x 9719 columns]


In [62]:

def Top_Recommendation(title):
    return movie_cos_sim_df[title].sort_values(ascending=False)[:6]
    

In [63]:
Top_Recommendation('Godfather, The (1972)')

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.821773
Goodfellas (1990)                            0.664841
One Flew Over the Cuckoo's Nest (1975)       0.620536
Star Wars: Episode IV - A New Hope (1977)    0.595317
Fargo (1996)                                 0.588614
Name: Godfather, The (1972), dtype: float64